##Algoritmo de previsão de preços de ações utilizando rede neural *Long short-term memory*

Utilizando a linguagem Python, desenvolver um programa para prever o preço de ações utilizando *Machine Learning*, mais especificamente uma Arquitetura de Redes Neurais *Long short-term memory (LSTM)*. Para esse projeto, foi utilizado o histórico de preços da ação da Microsoft obtido através da blbioteca *yfinance*.




In [19]:
# Bibliotecas utilizadas
import pandas as pd
import yfinance as yf
from datetime import date, timedelta

In [20]:
#Obtendo o conjunto dos dados
today = date.today()

endDate = today.strftime("%Y-%m-%d")
startDate = today - timedelta(days=5000)
startDateF = startDate.strftime("%Y-%m-%d")

data = yf.download('MSFT', start=startDateF, end=endDate, progress=False)

In [21]:
#Verificando as primeiras cinco linhas
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-08-25,27.610001,27.840000,27.459999,27.660000,20.691088,51381300
2008-08-26,27.580000,27.719999,27.170000,27.270000,20.399338,44774400
2008-08-27,27.340000,27.790001,27.129999,27.559999,20.616282,33975300
2008-08-28,27.610001,28.010000,27.600000,27.940001,20.900537,48372600
2008-08-29,27.680000,27.780001,27.290001,27.290001,20.414305,50735500


Análise Exploratória dos dados

In [22]:
#Verificando as informações dos objetos das colunas
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3447 entries, 2008-08-25 to 2022-05-03
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3447 non-null   float64
 1   High       3447 non-null   float64
 2   Low        3447 non-null   float64
 3   Close      3447 non-null   float64
 4   Adj Close  3447 non-null   float64
 5   Volume     3447 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 188.5 KB


In [24]:
#Checar se existe valores NA nas colunas dos dados
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [25]:
#Estatísticas descritivas das colunas
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,3447.000000,3447.000000,3447.000000,3447.000000,3447.000000,3.447000e+03
mean,84.956748,85.775982,84.092930,84.976107,80.499319,4.174088e+07
std,81.167039,81.968905,80.282600,81.166119,82.481501,2.519712e+07
min,15.200000,15.620000,14.870000,15.150000,11.487971,7.425600e+06
25%,28.805000,29.025001,28.510000,28.785001,23.014761,2.479955e+07
50%,46.910000,47.430000,46.529999,47.000000,41.715446,3.484970e+07
75%,108.559998,109.530003,107.475002,108.445000,104.291428,5.166245e+07
max,344.619995,349.670013,342.200012,343.109985,342.402008,3.193179e+08


In [28]:
#Verificar a correlação de todas as colunas com a coluna alvo "Close"
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    0.999978
Low          0.999899
High         0.999888
Open         0.999794
Volume      -0.341257
Name: Close, dtype: float64
